In [4]:
import numpy as np
import scipy as sp
import pandas as pd
import statsmodels.api as sm
import timeit
import pprint
from pandas.io.json import json_normalize
import re
import json
import pickle
import gensim
import datetime
import time
import sys
import fastparquet
import sklearn
from sklearn.preprocessing import normalize

from queue import Queue
from threading import Thread

import multiprocessing
from multiprocessing import cpu_count

import random

import os
os.chdir('/mnt/t48/bighomes-active/sfeng/patentdiffusion/')

seed = 3

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(level=logging.INFO)
logger.addHandler(logging.FileHandler('Logs/LDA_{0}.log'.format(datetime.datetime.now().\
                                                            strftime("%Y-%m-%d"), 'a')))
print = logging.info
print('good day to you madam fiona')
print('started')
print(datetime.datetime.now())

INFO:root:good day to you madam fiona
INFO:root:started
INFO:root:2018-07-13 13:23:12.141216


Code based on:
- https://sfengc7.stern.nyu.edu:8888/edit/patentdiffusion/Scripts/LDAModel_Sampled_0605.py
- https://sfengc7.stern.nyu.edu:8888/notebooks/patentdiffusion/201804KnowledgeSpilloversRep/2018-03-P1/0c-RefitMissingML.ipynb

### First clean abstracts

In [20]:
# Merge existing with missing
patabs = fastparquet.ParquetFile('PatBib/patabs7615_us.parq').to_pandas()
pa = pd.read_pickle("PatBib/missing_abs_text_stemmed_0312.pkl")
patabs = patabs.append(pa, ignore_index = True)
del(pa)
print(len(patabs))

# Convert to integer
patabs["patent"] = patabs["Patent"].astype(int)
patabs = patabs.drop("Patent",1)

# Record duplicates
patabs = patabs.drop_duplicates("patent")
print(len(patabs))

patdup = patabs.loc[patabs.duplicated("abs_stemmed"), "patent"]
patdup.to_pickle("RawData/Cleaned/duplicate_pattext_0712.pkl")

# Remove duplicates
patabs = patabs.drop_duplicates("abs_stemmed")
print(len(patabs))

2943842
2943841
2575725


In [18]:
fastparquet.write("RawData/Cleaned/patabs7615_us_no_dup.parq", patabs, compression="GZIP")

In [22]:
del(patabs, patdup)

___

## Fit LDA

In [3]:
def gensim_corpus(pattext_column):
    texts = [t.split() for t in pattext_column.tolist()]
    dictionary = gensim.corpora.Dictionary(texts)
    # Resulting dictionary is around 41000, keep to 40000
    dictionary.filter_extremes(no_below=20, no_above=0.1, keep_n = 40010)
    dictionary.filter_n_most_frequent(10)
    dictionary.compactify()
    corpus = [dictionary.doc2bow(text) for text in texts]
    return dictionary, corpus

def convert_corpus(pattext_column, samp_dict):
    texts = [t.split() for t in pattext_column.tolist()]
    corpus = [samp_dict.doc2bow(text) for text in texts]
    return corpus
    

def convert_topic_vector(row):
    tv = np.zeros(60)
    for i in iter(pattopics[row]):
        # Pattopics is (3, 0.05). So replace the i[0] = topic 3 with proportion i[1] = 0.05
        tv[i[0]] = i[1]
    return tv

In [1]:
# Part 1: Fit LDA Model
numtopics = 60

# # Abstracts
# patabs = fastparquet.ParquetFile("RawData/Cleaned/patabs7615_us_no_dup.parq").to_pandas(["abs_stemmed"])\
# .sample(frac = 0.5, random_state = seed)["abs_stemmed"]
# print(len(patabs))                

# # Generate and store dictionary & corpus
# print("generate and store dictionary & corpus")
# starttime = datetime.datetime.now()
# dict_samp, corpus_samp = gensim_corpus(patabs)

# # Filter extremes
# print("Dictionary: filter extremes")
# dict_samp.filter_extremes(no_below=20, no_above=0.1)

# dict_samp.save('DataStore/2018-07-P2/ML/lda_dict_0712.dict')
# gensim.corpora.MmCorpus.serialize('DataStore/2018-07-P2/ML/lda_corpus_0712.mm', corpus_samp)

# endtime = datetime.datetime.now()
# print(endtime)
# print("dict & corpus creation ended")
#----------#
# # Generate and store gensim LDA
# print("fit lda")
# print(datetime.datetime.now())

# print("Load dictionary & corpus")
# dict_samp = gensim.corpora.Dictionary.load('DataStore/2018-07-P2/ML/lda_dict_0712.dict')
# corpus_samp = gensim.corpora.MmCorpus('DataStore/2018-07-P2/ML/lda_corpus_0712.mm')


# print(datetime.datetime.now())
# print("Fitting")
# lda = gensim.models.LdaModel(corpus_samp, id2word = dict_samp, num_topics = numtopics, random_state = seed,
#                                 minimum_probability = 0.0)
# lda.save("DataStore/2018-07-P2/ML/lda_samp_60_0712")

# # Delete the corpus generated by the sample
# del(corpus_samp, dict_samp,lda)

# print("Ended Fit")
# print(datetime.datetime.now())

#---------#
# Part 2: Convert topics to vectors

# Create new corpus from sample dictionary
numtopics = 60

# Load abstract, dictionary, LDA model
patabs = fastparquet.ParquetFile("RawData/Cleaned/patabs7615_us_no_dup.parq").to_pandas(["abs_stemmed"])["abs_stemmed"]
dict_samp = gensim.corpora.Dictionary.load('DataStore/2018-07-P2/ML/lda_dict_0712.dict')
lda = gensim.models.ldamodel.LdaModel.load('DataStore/2018-07-P2/ML/lda_samp_60_0712')

# Convert full abstract corpus
conv_corpus = convert_corpus(patabs, dict_samp)

gensim.corpora.MmCorpus.serialize('DataStore/2018-07-P2/ML/lda_corpus_full_0712.mm', conv_corpus)

# If it breaks
# conv_corpus = gensim.corpora.MmCorpus('DataStore/2018-07-P2/ML/lda_corpus_full_0712.mm')

# Save as topic
pattopics = lda[conv_corpus]

# Delete all files
del(lda, conv_corpus, patabs, dict_samp)

print("multiprocess topic to vec")
starttime = datetime.datetime.now()
print(starttime)
num_workers = 4    
pool = multiprocessing.Pool(processes = multiprocessing.cpu_count()-1, maxtasksperchild=1000)

out = []
r = pool.map_async(convert_topic_vector, range(len(pattopics)), callback=out.append)  
r.wait()

out = out[0]

pool.close()
pool.terminate()

endtime = datetime.datetime.now()
print(endtime)
print("topic to vec ended")

cols = ["{0}".format(i) for i in range(numtopics)]
out = pd.DataFrame(out, columns = cols)

fastparquet.write("DataStore/2018-07-P2/ML/tm_0712.parq", out, compression = "GZIP")
    
# Part 3: Convert topic matrix to normalized topic matrix
# This won't make the zero topics non-zero, only renormalies the 
print("renormalize topic mat")
starttime = datetime.datetime.now()
print(starttime)
# out = fastparquet.ParquetFile('DataStore/2017-06/tm_0603.parq').to_pandas()

out2 = normalize(out, norm = "l1", axis = 1)
out2 = pd.DataFrame(out2, columns = cols)

# Save to pickle
fastparquet.write("DataStore/2018-07-P2/ML/ldavecs_pats_0712.parq", out2, compression = "GZIP")

endtime = datetime.datetime.now()
print(endtime)
print("renormalization ended")

__________

## Loading matrices using dask

In [6]:
import dask.dataframe as dd

dv = dd.read_parquet("DataStore/2018-07-P2/ML/docvecs_pats_0712.parq").values.compute()

In [8]:
dv[1]

array([ 1.63650155e-01, -3.88033455e-03, -8.33285302e-02, -3.37433219e-01,
       -2.84327924e-01, -2.33318098e-02,  6.59138560e-02, -2.24007308e-01,
       -1.01193190e-02,  9.62409079e-02, -5.00944592e-02, -5.05889617e-02,
       -7.76260048e-02, -7.03641102e-02, -5.40510789e-02, -4.65533212e-02,
       -7.73217753e-02, -1.35165066e-01,  2.59812474e-01, -5.28078787e-02,
        1.26593217e-01, -1.40859634e-01, -3.55752409e-02,  2.19237253e-01,
       -4.96096238e-02, -3.98100428e-02, -5.60732298e-02, -1.77158322e-02,
       -1.07153758e-01,  7.30056986e-02,  1.73478767e-01, -1.95621755e-02,
        3.78771387e-02, -1.12942904e-01,  2.69509722e-02,  1.16238251e-01,
        1.26931027e-01, -5.77591099e-02, -1.00283951e-01, -4.21032906e-01,
       -1.51542351e-01,  2.51701921e-01,  2.12049007e-01,  3.19784544e-02,
       -1.65253907e-01,  8.97208452e-02,  2.35672757e-01,  1.84998810e-01,
       -3.35111879e-02, -1.14441365e-01, -3.89985368e-02, -1.06185853e-01,
        2.44154215e-01,  